In [1]:
import numpy as np
import struct #reads binary info
import gzip #reads the file
import time #keeps track of time

#importing the reusable library
from pynnet import Sequential, DenseLayer, ActivationLayer, sigmoid, sigmoid_prime, softmax, categorical_cross_entropy, categorical_cross_entropy_prime
#reading the files
def read_idx(filename):
    """Reads an IDX file and returns its content as a numpy array."""
    try:
        with gzip.open(filename, 'rb') as f:
            zero, data_type, dims = struct.unpack('>HBB', f.read(4))
            shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
            return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    except gzip.BadGzipFile:
        with open(filename, 'rb') as f:
            zero, data_type, dims = struct.unpack('>HBB', f.read(4))
            shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
            return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

def to_categorical(y, num_classes=10):
    """Converts an integer class vector to a one-hot encoded matrix."""
    return np.eye(num_classes)[y]

#1-Loading the data
print("Loading data from local files...")
x_train_raw = read_idx('train-images-idx3-ubyte')
y_train_raw = read_idx('train-labels-idx1-ubyte')
x_test_raw = read_idx('t10k-images-idx3-ubyte')
y_test = read_idx('t10k-labels-idx1-ubyte')
print("Data loaded successfully.")

#2-Preprocessing the data
# Normalize pixel values and flatten images
x_train = x_train_raw.astype('float32') / 255
x_train = x_train.reshape(len(x_train), 28 * 28)
# One hot encoding the labels
y_train = to_categorical(y_train_raw, 10)

# Use a smaller subset for faster training
x_train_small = x_train[:1000]
y_train_small = y_train[:1000]

#3-Model and train
model = Sequential()
model.add(DenseLayer(28 * 28, 128))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
#The final layer uses softmax
model.add(DenseLayer(128, 10))
model.add(ActivationLayer(softmax, None)) #no prime because the value depends on all neurons

#4-Compile and run
print("\nStarting model training with Softmax and Cross-Entropy...")
model.compile(loss=categorical_cross_entropy, loss_prime=categorical_cross_entropy_prime, learning_rate=0.1)
start_time = time.time()

model.fit(x_train_small, y_train_small, epochs=50)
end_time = time.time()
print(f"Training complete in {end_time - start_time:.2f} seconds.")

#5-Evaluate model
x_test_processed = x_test_raw.astype('float32') / 255
x_test_processed = x_test_processed.reshape(len(x_test_processed), 28 * 28)

predictions = model.predict(x_test_processed[:200])
correct_predictions = sum(1 for i in range(200) if np.argmax(predictions[i]) == y_test[i])
accuracy = (correct_predictions / 200) * 100

print(f"\nAccuracy on first 200 test samples: {accuracy:.2f}%")
print("\n--- Example Predictions (First 10) ---")
for i in range(10):
    predicted_digit = np.argmax(predictions[i])
    true_digit = y_test[i]
    print(f"Prediction: {predicted_digit}, True Value: {true_digit}")

Loading data from local files...
Data loaded successfully.

Starting model training with Softmax and Cross-Entropy...
Epoch 1/50, Loss: 0.09826
Epoch 11/50, Loss: 0.00094
Epoch 21/50, Loss: 0.00035
Epoch 31/50, Loss: 0.00021
Epoch 41/50, Loss: 0.00015
Epoch 50/50, Loss: 0.00012
Training complete in 52.26 seconds.

Accuracy on first 200 test samples: 95.00%

--- Example Predictions (First 10) ---
Prediction: 7, True Value: 7
Prediction: 2, True Value: 2
Prediction: 1, True Value: 1
Prediction: 0, True Value: 0
Prediction: 4, True Value: 4
Prediction: 1, True Value: 1
Prediction: 4, True Value: 4
Prediction: 9, True Value: 9
Prediction: 2, True Value: 5
Prediction: 9, True Value: 9
